In [ ]:
import os
if "models" not in os.listdir("."):
    os.chdir("..")

In [ ]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [ ]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf",
                                         from_type="gemma",
                                         load_eager=True
                                         )

In [ ]:
!mkdir -p data
import pandas as pd
df_adv = pd.read_csv("data/adv.csv")
format_prompt = """<start_of_turn>user\n
{}\n
<start_of_turn>model\n
{}"""
# offset = 1
# df_do = df.apply(lambda x: format_response.format(x['goal'], x['target']), axis=1)
# prompts_harmful = df.apply(lambda x: format_prompt.format(x['goal'], "")[:-offset], axis=1).to_list()[:100]
prompts_harmful = df_adv.apply(lambda x: format_prompt.format(x['goal'], ""), axis=1).to_list()[:100]
dataset_jail = pd.read_csv("data/jail.csv").apply(lambda x: x["Goal"], axis=1).to_list()
prompts_jail = [format_prompt.format(x, "") for x in dataset_jail]
import datasets
# https://colab.research.google.com/drive/1a-aQvKC9avdZpdyBn4jgRQFObTPy1JZw
hf_path = 'tatsu-lab/alpaca'
dataset = datasets.load_dataset(hf_path)
# filter for instructions that do not have inputs
prompts_harmless = []
for i in range(len(dataset['train'])):
    if len(prompts_harmless) >= len(prompts_harmful):
        break
    if dataset['train'][i]['input'].strip() == '':
        # prompts_harmless.append(format_prompt.format(dataset['train'][i]['instruction'], "")[:-offset])
        prompts_harmless.append(format_prompt.format(dataset['train'][i]['instruction'], ""))

# ds = datasets.load_dataset("MBZUAI/LaMini-instruction", split="train", streaming=True)
# prompts_harmless = []
# for _, text in zip(range(100), ds):
#     prompts_harmless.append(format_prompt.format(text["instruction"], ""))

# ds = datasets.load_dataset("nev/openhermes-2.5-phi-format-text", split="train", streaming=True)
# prompts_harmless = []
# for _, text in zip(range(100), ds):
#     text = text["text"]
#     text = "".join(text.partition("<|assistant|>\n")[:2])
#     prompts_harmless.append(text)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [ ]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
import jax

tokens = tokenizer.batch_encode_plus(prompts_harmful + prompts_harmless,
                                     return_tensors="np",
                                     padding="max_length",
                                     truncation=True,
                                     max_length=128,
                                     return_attention_mask=True)
token_array = jnp.asarray(tokens["input_ids"])
token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")
inputs = llama.inputs.from_basic_segments(token_array)

In [ ]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)
_, resids = get_resids_call(inputs)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
residiffs = []
last_resids = []
for i, resid in enumerate(sorted(resids, key=lambda x: int(x.tag.rpartition("_")[-1]))):
    resid = resid.value.unwrap("batch", "seq", "embedding")
    indices = jnp.asarray(tokens["attention_mask"].sum(1, keepdims=True))[..., None] - 1
    last_resid = jnp.take_along_axis(resid, indices, 1)[:, 0]
    last_resid = last_resid.reshape(2, -1, last_resid.shape[-1])
    last_resids.append(last_resid)
    last_resid = last_resid.mean(1)
    residiff = np.array(last_resid[0] - last_resid[1])
    residiff = residiff / np.linalg.norm(residiff)
    residiffs.append(residiff)
matmuls = np.matmul(residiffs, np.transpose(residiffs))
norms = np.linalg.norm(residiffs, axis=-1) + 1e-10
plt.title("Correlations between layers' refusal vectors")
plt.imshow(matmuls / norms[:, None] / norms[None, :], vmin=0, vmax=1)
plt.colorbar()
plt.show()

In [ ]:
from micrlhf.utils.vector_storage import save_and_upload_vector, download_vector
from micrlhf.utils.load_sae import get_nev_it_sae_suite
from micrlhf.utils.ito import grad_pursuit


sae_k = 8
threshold = 0.15
interesting_features = set()
for layer_sae in range(8, 16):
    dictionary = get_nev_it_sae_suite(layer_sae)["W_dec"]
    dictionary = dictionary / np.linalg.norm(dictionary, axis=1, keepdims=True)
    for layer in range(1, len(residiffs)):
        vector_name = f"gemma-refusal-l{layer}"
        vector = residiffs[layer]
        vector = vector / np.linalg.norm(vector)
        try:
            save_and_upload_vector(vector_name, vector)
        except FileExistsError:
            pass
        weights, recon = grad_pursuit(vector, dictionary, sae_k, pos_only=True)
        w, i = jax.lax.top_k(jnp.abs(weights), sae_k)
        print(f"Layer {layer} -> Layer {layer_sae}: {[(int(a), float(b)) for a, b in zip(i, w)]}")
        for f, u in zip(i, w):
            if u < threshold:
                continue
            interesting_features.add((layer_sae, int(f)))
    print()

In [ ]:
from micrlhf.llama import LlamaAttention
influences = {}
influences_attn = {}
all_feat_directions = {}

def prop_sae(feat_source, sae_mid, feat_target):
    strengths = (feat_source @ sae_mid["W_enc"] * jax.nn.softplus(sae_mid["s_gate"]) * sae_mid["scaling_factor"]) * (sae_mid["W_dec"] @ feat_target)
    # dic = sae_mid["W_dec"]
    # dic = dic
    # dic_norm = jnp.linalg.norm(dic, axis=1)
    # strengths = ((dic @ feat_source) * (dic @ feat_target)) / (dic_norm ** 2)
    return strengths

n_top = 8
strength_thresh = 0.016
for i, (layer_source, feature_source) in enumerate(tqdm(all_feats)):
    for j, (layer_target, feature_target) in enumerate(all_feats):
        if layer_target <= layer_source:
            continue
        feat_source = get_nev_it_sae_suite(layer_source)["W_dec"][feature_source]
        target_sae = get_nev_it_sae_suite(layer_target)
        feat_target = (target_sae["W_enc"] * jax.nn.softplus(sae_mid["s_gate"]) * sae_mid["scaling_factor"]).T[feature_target]
        # feat_target = target_sae["W_dec"][feature_target]
        # feat_source = feat_source / np.linalg.norm(feat_source)
        # feat_target = feat_target / np.linalg.norm(feat_target)

        edge_key = ((layer_source, feature_source), (layer_target, feature_target))
        all_feat_directions[edge_key] = (np.array(feat_source), np.array(feat_target))
        influencers = []
        for layer_mid in range(layer_source , layer_target):
            sae_mid = get_nev_it_sae_suite(layer_mid, label="transcoder")
            strengths =prop_sae(feat_source, sae_mid, feat_target)
            top_effects, top_feats = jax.lax.top_k(jnp.abs(strengths), n_top)
            for f in top_feats:
                influencers.append((float(strengths[f]), layer_mid, int(f)))
        # influencers.sort(key=lambda x: abs(x[0]))
        # display(influencers)
        influencers = [(s, l, f) for s, l, f in influencers if abs(s) > strength_thresh]  # vibes
        influences[edge_key] = influencers
        
        influencers_attn = []
        for layer_mid in range(layer_source, layer_target):
            sae_mid = get_nev_it_sae_suite(layer_mid, label="attn_out")
            attn_layer = llama.select().at_instances_of(LlamaAttention).get_sequence()[layer_mid]
            v_linear = attn_layer.input_to_value.select().at_instances_of(pz.nn.Linear).get_sequence()[0].weights.value
            o_linear = attn_layer.attn_value_to_output.select().at_instances_of(pz.nn.Linear).get_sequence()[0].weights.value
            fs = jnp.einsum("a,abc,rbco->o", feat_source, v_linear.unwrap("embedding", "kv_heads", "projection"), o_linear.unwrap("q_rep", "kv_heads", "projection", "embedding"))
            strengths = prop_sae(fs, sae_mid, feat_target)
            top_effects, top_feats = jax.lax.top_k(jnp.abs(strengths), n_top)
            for f in top_feats:
                influencers_attn.append((float(strengths[f]), layer_mid, int(f)))
        # influencers_attn.sort(key=lambda x: abs(x[0]))
        influencers_attn = [(s, l, f) for s, l, f in influencers_attn if abs(s) > strength_thresh]
        influences_attn[edge_key] = influencers_attn

In [ ]:
interesting_attn_outs = set()
for key, influencers in influences_attn.items():
    for _, layer, feature in influencers:
        interesting_attn_outs.add((layer, feature))

In [ ]:
from tqdm.auto import tqdm
import requests
import numpy as np
import textwrap
import os
import json

def norm(x):
    x = np.array(x)
    x = x / max(1e-3, x.max())
    return x

feat_dir = "data/feature_explanations"
os.makedirs(feat_dir, exist_ok=True)
feat_explanations = {}
max_len = 72
linebreak = 12
for t, l, f in tqdm(set(("R", l, f) for l, f in interesting_features) | set(("A", l, f) for l, f in interesting_attn_outs)):
    filename = f"{feat_dir}/explanation_{t}_{l}_{f}.json"
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            data = json.load(file)
    else:
        response = requests.get(f"https://datasets-server.huggingface.co/rows?dataset=kisate-team%2Fgemma-2b-suite-explanations&config=l{l}{'' if t == 'R' else '_attn_out' if t == 'A' else 1/0}&split=train&offset={f}&length=1")
        data = response.json()
        with open(filename, 'w') as file:
            json.dump(data, file)
    
    if not data or set(data.keys()) == {"error"}:
        print("skipping", l, f, data)
        continue
    
    feat_info = data["rows"][0]["row"]
    criterion = norm(feat_info["scale_tuning"]["selfsims"][-2]) * 0.4 - 0.01 * norm(feat_info["scale_tuning"]["entropy"])
    scale = feat_info["scale_tuning"]["scales"][np.argmax(criterion[10:]) + 10]
    index = np.searchsorted(feat_info["generations"]["scales"], scale)
    texts = feat_info["generations"]["texts"]
    # display((l, f, scale, texts[index], texts))
    print((t, l, f, scale, repr(texts[index])))
    unprocessed = texts[index]
    processed = unprocessed.partition("<eos>")[0].partition('"')[0].strip()
    if len(processed) > max_len:
        processed = processed[:max_len] + "..."
    processed = f"{t}{l}.{f}: {processed}"
    feat_explanations[f"{t}{l}.{f}"] = textwrap.fill(processed, width=linebreak)

In [ ]:
from IPython.display import Image as IImage
from tqdm.auto import tqdm
import graphviz
import random

all_feats = list(interesting_features)
graph = graphviz.Digraph(graph_attr={"overlap": "scale", "splines": "ortho", "engine": "sfdp"})
edges_out = set()
direct_thresh = 0.3
letter_thresholds = {"T": 0.025, "A": 0.025}
included_nodes = set()
s_scales = {"T": 0.05, "A": 0.05}
penwidth_edge = 2
penwidth_direct = 2
direct_scale = 0.5
for i, (layer_source, feature_source) in enumerate(tqdm(all_feats)):
    for j, (layer_target, feature_target) in enumerate(all_feats):
        if layer_source == layer_target or layer_source > layer_target:
            continue
        name_source = f"R{layer_source}.{feature_source}"
        name_target = f"R{layer_target}.{feature_target}"
        
        edge_key = ((layer_source, feature_source), (layer_target, feature_target))
        a, b = all_feat_directions.get(edge_key, (np.zeros(0), np.zeros(0)))
        draw_direct_edge = a @ b > direct_thresh
        if draw_direct_edge:
            graph.edge(name_source, name_target, color="black", penwidth=f"{penwidth_direct * abs(a @ b) / direct_scale:.4f}")
        influencers = influences.get(edge_key, [])
        influencers_attn = influences_attn.get(edge_key, [])
        
        influencers = [("T", s, l, f) for s, l, f in influencers] + [("A", s, l, f) for s, l, f in influencers_attn]
        influencers = [(t, s, l, f) for t, s, l, f in influencers if abs(s) > letter_thresholds[t]]
        has_edge = draw_direct_edge or influencers
        if has_edge:
            graph.node(name_source, label=feat_explanations[name_source], shape="box")
            graph.node(name_target, label=feat_explanations[name_target], shape="box")
            included_nodes.add(name_source)
            included_nodes.add(name_target)
        
        for letter, s, l, f in influencers:
            edge = f"{letter}{l}.{f}"
            edge_name = feat_explanations.get(edge, edge)
            edge_out = edge, name_target
            gen = random.Random("->".join(edge_out))
            r, g, b = gen.randint(0, 255), gen.randint(0, 255), gen.randint(0, 255)
            color = f"#{r:02X}{g:02X}{b:02X}"
            graph.node(edge, label=edge_name, shape=dict(A="ellipse", T="egg")[letter], color=color)
            graph.edge(name_source, edge, color=color, penwidth=f"{penwidth_edge * abs(s) / s_scales[letter]:.4f}")
            if edge_out in edges_out:
                continue
            edges_out.add(edge_out)
            graph.edge(*edge_out, color=color, penwidth=str(penwidth_edge))
graph.render("data/refusal-gemma-2b", format="png", cleanup=True)
print("Included:", len(included_nodes), "Total:", len(all_feats))
IImage("data/refusal-gemma-2b.png")